In [11]:
import ast
import json
from datetime import datetime
from pathlib import Path

import pandas as pd
import torch
from PIL import Image
from megadetector.detection.run_detector_batch import load_and_run_detector_batch

from helpers import Deepfaune, crop_normalized_bbox_square, predict_batch, class_names

#### Settings and init

In [ ]:
images_directory = '../test_images'  # user input
BATCH_SIZE = 30  # user input?
BATCH_SIZE_MD = 14  # user input?
N_CORES = 12  # user input?
checkpoint_path = 'model/deepfaune_polish_lr4_checkpoint.pt' # user input in advanced?

In [3]:
# images
image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp'}
directory = Path(images_directory)
images_paths = [
    str(p) for p in directory.rglob("*")
    if p.suffix.lower() in image_extensions
]

#### Detection

In [4]:
results = load_and_run_detector_batch("MDV5A", images_paths, confidence_threshold=0.2, batch_size=BATCH_SIZE_MD, n_cores=N_CORES)

Model v5a.0.1 already exists and is valid at /tmp/megadetector_models/md_v5a.0.1.pt
PyTorch reports 1 available CUDA devices
GPU available: True
PyTorch reports 1 available CUDA devices
GPU available: True


/media/plk/TOSHIBA EXT/animal_classification_app/venv/lib/python3.12/site-packages/yolov5/utils/general.py:31: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg


Loading PT detector with compatibility mode classic
Loaded image size 1280 from model metadata
Using model stride: 64
PTDetector using device cuda:0


Fusing layers... 
Fusing layers... 
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs


Loaded model in 1.76 seconds


100%|██████████| 8/8 [00:19<00:00,  2.38s/it]


In [ ]:
images_paths_sec, categories, confs, bboxes, n_animals = [], [], [], [], []

for result in results:
    images_paths_sec.append(result.get('file'))
    
    detections = result.get('detections')
    if detections:
        detection = max(detections, key=lambda d: d["conf"])
        categories.append(detection.get('category'))
        confs.append(detection.get('conf'))
        bboxes.append(detection.get('bbox'))
        n_animals.append(len(detections))
    else:
        categories.append(None)
        confs.append(None)
        bboxes.append(None)
        n_animals.append(0)

results_df = pd.DataFrame({'image_path': images_paths_sec, 'category': categories, 'conf': confs, 'bbox': bboxes, 'n_animals': n_animals})
results_df.to_csv(f'megadetector_results.csv')

with open('megadetector_raw_results.json', 'w') as output_file:
    json.dump(results, output_file, indent=4)

In [14]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

#### Classification

In [15]:
# classifier model
model_wrapper = Deepfaune(checkpoint_path)
classifier = model_wrapper.model.base_model
classifier.to('cuda')
transforms = model_wrapper.transforms

Using model in resolution 476x476
CUDA available


In [18]:
images = pd.read_csv('megadetector_results.csv', index_col=0)
total=len(images)
images['bbox'] = images["bbox"].apply(
    lambda b: ast.literal_eval(b) if isinstance(b, str) else None)

batch = []
paths = []
results = pd.DataFrame({'image': [], 'detected_animal': [], 'confidence': []})
ith_image = 0

for _, row in images.iterrows():
    ith_image += 1
    print(f'{100*ith_image/total:.1f}%: {ith_image} / {total}', end='\r')
    image_path = row['image_path']

    # only animals
    category = row['category']
    if category != 1:
        results.loc[len(results)] = [image_path, 'empty', 0]
        continue

    # image
    try:
        image = Image.open(image_path).convert("RGB")
        cropped_image = crop_normalized_bbox_square(image, row['bbox'])
    except Exception as e:
        # print(f'Error in image {image_path}: {e}')
        continue

    paths.append(image_path)
    batch.append(cropped_image)

    # run classifier every N images (e.g. 32)
    if len(batch) == BATCH_SIZE:
        preds = predict_batch(classifier, batch, transforms, class_names)
        # if confidence (prediction[0][1]) is less than 0.1, classify as other
        detections = [
            prediction[0][0] if prediction[0][1] > 0.1 else 'other' for prediction in preds]
        confs = [prediction[0][1] for prediction in preds]

        batch_results = pd.DataFrame(
            {'image': paths, 'detected_animal': detections, 'confidence': confs})
        results = pd.concat([results, batch_results], ignore_index=True)
        # if confidence less than threshold: other
        batch = []
        paths = []

if len(batch) > 0:
    preds = predict_batch(classifier, batch, transforms, class_names)
    detections = [
        prediction[0][0] if prediction[0][1] > 0.1 else 'other' for prediction in preds]
    confs = [prediction[0][1] for prediction in preds]

    batch_results = pd.DataFrame(
        {'image': paths, 'detected_animal': detections, 'confidence': confs})
    results = pd.concat([results, batch_results], ignore_index=True)

In [17]:
now = datetime.now().strftime('%Y_%m_%d_%H_%M')
results.to_csv(f'results_{now}.csv')